In [2]:
import pandas as pd
import numpy as np

In [3]:
books = pd.read_csv("Books.csv")
users = pd.read_csv("Users.csv")
ratings = pd.read_csv("Ratings.csv")

C:\Users\admin\AppData\Local\Temp\ipykernel_5552\2906776592.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Books.csv")


EDA

In [4]:
print(books.shape, ratings.shape, users.shape)
print(books.isnull().sum())
print(ratings.isnull().sum())
print(users.isnull().sum())

(271360, 8) (1149780, 3) (278858, 3)
ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64
User-ID          0
Location         0
Age         110762
dtype: int64


In [5]:
books['Book-Title'] = books['Book-Title'].str.strip()
books['Book-Author'] = books['Book-Author'].str.strip()
# Convert 'Year-Of-Publication' to numeric (errors='coerce' turns bad data to NaN)
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'], errors='coerce')

books = books[books['Year-Of-Publication'].between(1800, 2025)]

books.dropna(inplace=True)

In [6]:
users.drop(columns=['Age'], inplace=True)
# Optional: Remove implicit feedback (rating = 0)
ratings = ratings[ratings['Book-Rating'] > 0]

In [7]:
worthwhile_books = ratings['ISBN'].value_counts()[ratings['ISBN'].value_counts() >= 30].index

In [8]:
active_users = ratings['User-ID'].value_counts()[ratings['User-ID'].value_counts() >= 30].index


In [9]:
print(worthwhile_books.shape)
print(active_users.shape)

(1148,)
(2413,)


In [10]:
print("Books shape:", books.shape)
print("Ratings shape:", ratings.shape)
print("Users shape:", users.shape)

Books shape: (266721, 8)
Ratings shape: (433671, 3)
Users shape: (278858, 2)


In [11]:
ratings_with_books = ratings.merge(books, on='ISBN')
ratings_full = ratings_with_books.merge(users, on='User-ID')

In [12]:
num_rating = ratings_with_books.groupby("Book-Title").count()["Book-Rating"].reset_index()
num_rating.rename(columns={"Book-Rating":"num-rating"}, inplace=True)
num_rating

,Book-Title,num-rating
0,!%@ (A Nutshell handbook),1
1,!Arriba! Comunicacion y cultura,1
2,"!Trato hecho!: Spanish for Real Life, Combined...",1
3,!Yo!,7
4,$14 In The Bank (Cathy Collection),1
...,...,...
133359,Ã?Â?bernachten mit Stil. Die schÃ?Â¶nsten Coun...,1
133360,Ã?Â?rger mit Produkt X. Roman.,3
133361,Ã?Â?sterlich leben.,1
133362,Ã?Â?stlich der Berge.,1


Popularity based Filtering

In [13]:
avg_rating = ratings_with_books.groupby("Book-Title")["Book-Rating"].mean().reset_index()
avg_rating.rename(columns={"Book-Rating": "avg-rating"}, inplace=True)
avg_rating

,Book-Title,avg-rating
0,!%@ (A Nutshell handbook),6.000000
1,!Arriba! Comunicacion y cultura,9.000000
2,"!Trato hecho!: Spanish for Real Life, Combined...",5.000000
3,!Yo!,8.142857
4,$14 In The Bank (Cathy Collection),9.000000
...,...,...
133359,Ã?Â?bernachten mit Stil. Die schÃ?Â¶nsten Coun...,8.000000
133360,Ã?Â?rger mit Produkt X. Roman.,7.000000
133361,Ã?Â?sterlich leben.,7.000000
133362,Ã?Â?stlich der Berge.,8.000000


In [14]:
new_df = num_rating.merge(avg_rating, on="Book-Title")
new_df

,Book-Title,num-rating,avg-rating
0,!%@ (A Nutshell handbook),1,6.000000
1,!Arriba! Comunicacion y cultura,1,9.000000
2,"!Trato hecho!: Spanish for Real Life, Combined...",1,5.000000
3,!Yo!,7,8.142857
4,$14 In The Bank (Cathy Collection),1,9.000000
...,...,...,...
133359,Ã?Â?bernachten mit Stil. Die schÃ?Â¶nsten Coun...,1,8.000000
133360,Ã?Â?rger mit Produkt X. Roman.,3,7.000000
133361,Ã?Â?sterlich leben.,1,7.000000
133362,Ã?Â?stlich der Berge.,1,8.000000


In [15]:
new_df = new_df[new_df['num-rating'] >= 250].sort_values('avg-rating', ascending=False)

In [16]:
new_df.shape

(27, 3)

In [17]:
popular_df = new_df.merge(books, on="Book-Title").drop_duplicates("Book-Title")

In [18]:
popular_df.shape

(27, 10)

In [19]:
popular_df.rename(columns={"num-rating": "num_rating", "avg-rating": "avg_rating"}, inplace=True)
popular_df = popular_df[["Book-Title", "Book-Author", "Image-URL-M", "num_rating", "avg_rating"]]

popular_df


,Book-Title,Book-Author,Image-URL-M,num_rating,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,277,9.043321
3,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,267,8.977528
11,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,315,8.936508
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,326,8.840491
16,"Tuesdays with Morrie: An Old Man, a Young Man,...",MITCH ALBOM,http://images.amazon.com/images/P/0385484518.0...,250,8.588000
19,The Secret Life of Bees,Sue Monk Kidd,http://images.amazon.com/images/P/0142001740.0...,406,8.477833
25,The Da Vinci Code,Dan Brown,http://images.amazon.com/images/P/0385504209.0...,494,8.439271
30,The Lovely Bones: A Novel,Alice Sebold,http://images.amazon.com/images/P/0316666343.0...,707,8.185290
31,The Red Tent (Bestselling Backlist),Anita Diamant,http://images.amazon.com/images/P/0312195516.0...,383,8.182768
32,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,http://images.amazon.com/images/P/0446672211.0...,295,8.142373


In [22]:
popular_df = popular_df.head(15)
popular_df

,Book-Title,Book-Author,Image-URL-M,num_rating,avg_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,277,9.043321
3,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,267,8.977528
11,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,315,8.936508
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,326,8.840491
16,"Tuesdays with Morrie: An Old Man, a Young Man,...",MITCH ALBOM,http://images.amazon.com/images/P/0385484518.0...,250,8.588000
19,The Secret Life of Bees,Sue Monk Kidd,http://images.amazon.com/images/P/0142001740.0...,406,8.477833
25,The Da Vinci Code,Dan Brown,http://images.amazon.com/images/P/0385504209.0...,494,8.439271
30,The Lovely Bones: A Novel,Alice Sebold,http://images.amazon.com/images/P/0316666343.0...,707,8.185290
31,The Red Tent (Bestselling Backlist),Anita Diamant,http://images.amazon.com/images/P/0312195516.0...,383,8.182768
32,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,http://images.amazon.com/images/P/0446672211.0...,295,8.142373


Collaborative Filtering

In [23]:
filtered_rating = ratings_with_books[ratings_with_books["User-ID"].isin(active_users)]


In [24]:
active_users.shape

(2413,)

In [25]:
y = filtered_rating.groupby("Book-Title").count()["Book-Rating"] >= 30
active_books = y[y].index

In [26]:
active_books.shape

(376,)

In [27]:
final_ratings = filtered_rating[filtered_rating["Book-Title"].isin(active_books)]

In [28]:
final_ratings.shape

(19571, 10)

In [29]:
final_ratings.drop_duplicates().shape

(19571, 10)

In [30]:
pt = final_ratings.pivot_table(index="Book-Title", columns="User-ID", values="Book-Rating")

In [31]:
pt.fillna(0, inplace=True)

In [32]:
pt.shape

(376, 2084)

In [33]:
pt

User-ID,183,242,254,507,638,882,1025,1211,1424,1435,...,277427,277639,277965,278026,278137,278188,278356,278418,278582,278633
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Case of Need,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wish You Well,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Wizard and Glass (The Dark Tower, Book 4)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Cosine Similarity

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
similarity_score = cosine_similarity(pt)

In [42]:
def recommend(name):
    index = np.where(pt.index == name)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])), key= lambda x:x[1], reverse=True)[1:6]
    
    data = []
    for i in similar_items:
        items = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        items.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        data.append(items)

    return data

In [43]:
recommend('Wish You Well')

[['Postmortem',
  'Patricia Daniels Cornwell',
  'http://images.amazon.com/images/P/0380710218.01.MZZZZZZZ.jpg'],
 ['Chosen Prey',
  'John Sandford',
  'http://images.amazon.com/images/P/0425182878.01.MZZZZZZZ.jpg'],
 ['Unnatural Exposure',
  'Patricia Daniels Cornwell',
  'http://images.amazon.com/images/P/0399142851.01.MZZZZZZZ.jpg'],
 ['Heartbreaker',
  'Julie Garwood',
  'http://images.amazon.com/images/P/0671034006.01.MZZZZZZZ.jpg'],
 ['Midnight Bayou',
  'Nora Roberts',
  'http://images.amazon.com/images/P/0399148248.01.MZZZZZZZ.jpg']]

In [38]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [39]:
books.drop_duplicates('Book-Title')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271354,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993.0,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991.0,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004.0,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996.0,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [41]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_score,open('similarity_score.pkl','wb'))